<a href="https://colab.research.google.com/github/DJongstra/Information_Retrieval_Assignment_3/blob/main/IR_PlagiarismDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Setup
- Import all needed libraries
- Google Drive mount


In [ ]:
from google.cloud import storage
from google.colab import drive
drive.mount('/content/drive')

!pip install mmh3
!pip install snapy
!pip install xxhash
!pip install Random-Word-Generator

import numpy as np
import seaborn as sns
import pandas as pd
import string, re, random, xxhash, time
from snapy import MinHash, LSH


# 2. Similarity Analysis: Ground Truth
Preprocessing of a document


In [ ]:
# "We don't need to use a library, great!" -> ["we", "do", "not", "need", "to", "use", "a", "library", "great"]
def preprocess_document(document: str):
    doc = document.lower()  # lower case
    doc = doc.replace("n't", " not").replace("'ve", " have").replace("'s", "")  # rewrite contractions
    doc = re.sub(" [^ ]*&amp[^ ]*", "", doc)  # remove random "&amp" in text
    doc = doc.translate(str.maketrans('', '', string.digits))  # remove numbers?
    doc = re.sub(" +", " ", doc)  # remove double spaces
    doc = doc.translate(str.maketrans('', '', string.punctuation))  # remove ALL punctuation
    return doc.split()

Load the small article dataset

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/IR-Assignment-3/data/news_articles_small.csv', index_col=0)
print(df)

In [ ]:
df['article'].iloc[0]

All the articles in the small article dataset will be processed to a list of the terms in the articles. The words are lowercased and duplicates are removed by using a set (because order does not matter in this part of the analysis).

In [ ]:
articleList = []

for _, row in df.iterrows():
    terms = preprocess_document(row['article'])
    articleList.append(set(terms))
    
print(articleList[0])

Calculate the jaccard index between each two documents in the data set by dividing the length of the intersection with the length of the union of the two sets. Save the values to a list to use later.

In [ ]:
jaccardVals = []

for doc1idx in range(len(articleList)):
  doc1 = articleList[doc1idx]
  doc2idx = doc1idx + 1
  while doc2idx < len(articleList):
    doc2 = articleList[doc2idx]
    jaccard = len(doc1.intersection(doc2)) / len(doc1.union(doc2))
    jaccardVals.append(jaccard)
    doc2idx += 1

Plot the amount of values per bin, using a total of 50 bins.


In [ ]:
jaccardVals = np.array(jaccardVals)
sns.histplot(jaccardVals, bins=50)


The previous graph showed a peak in a small range of the possible similarities. To see the distribution in other ranges, we leave the peak values out.

From this it is clear that there are also values in the higher ranges, however there are not a lot.

In [ ]:
sns.histplot(jaccardVals[jaccardVals>0.2], bins=40)

# 3. LSH Implementation


## 3.1 Hash functions
The class HashFunction uses the **xxhash** library to hash previously hashed shingles (64 bit) from sketches. We use a fixed size key to create different hash functions h_0 to h_|M|. Even before reading our documents, we will generate a list of these hash functions based on a **seed**.

The class can also be used to hash shingles (list of strings) to a 64 bit value.

In [ ]:
# Hash function object that can be prepared
# If no salt is provided this function will be a normal xxhash
# If two hash functions use the same salt, they will also generate the same output for a given input
# We call it salt, as it is not secret, but deterministic
class HashFunction:
    def __init__(self, salt: int = None):
        self.salt = self.int_to_bytes(salt) if salt else b''  # store key

    # ["rose", "is", "a"] -> 189939623769124324
    def compute_strings(self, shingle: []):
        h = xxhash.xxh64() # no salt needed
        for word in shingle:
            h.update(word)
        return self.to_64_bit(h.digest())

    # (hashed shingle) 189939623769124324 ->  (rank) 134237347983861913
    def compute_int64(self, shingle: int):
        h = xxhash.xxh64(self.salt)
        h.update(self.int_to_bytes(shingle))
        return self.to_64_bit(h.digest())

    # convert 64 bit integer to 8 bytes
    def int_to_bytes(self, i: int):
        return int.to_bytes(i, length=8, byteorder='big', signed=False)

    # convert 16 byte hash digest (128 bit) to a 64 bit integer (8 bytes)
    def to_64_bit(self, digest: bytes):
        return int.from_bytes(digest[:8], byteorder='big', signed=False)


## 3.2 LSH functionality

After all documents are added, the set is static and no documents can be added after. However, we can compare new content to our existing set using query_content(doc, s).
We could implement insertion, but we won't need it for this assignment.


In [ ]:
# Basic functionality that the library as wel as our own implementation must handle
class LSHFunctionality:
    def __init__(self, n_gram, bands, rows, seed):
        self.n_gram = n_gram
        self.bands = bands
        self.rows = rows
        self.seed = seed
        self.signature_length = bands * rows
        self.original_documents = []

    # read directly from csv file
    def read_csv(self, csv_file: str):
        for _, row in pd.read_csv(csv_file, index_col=0).iterrows():
            self.original_documents.append(row['article'])


    # add documents as a list of strings
    def add_documents(self, documents: []):
        self.original_documents = documents

    # after adding documents, call compute to start the LSH
    def compute(self):
        raise Exception("virtual")

    # return all similarities >= s
    def get_all_similarities(self, s: float):
        raise Exception("virtual")

    # compare all docs to 'content' and return all where >= s
    def query_content(self, content: str, s: float):
        raise Exception("virtual")


## 3.3 Using a Library
We first start with implementing this functionality with the library

https://pypi.org/project/snapy/

In [ ]:
class LSHLibrary(LSHFunctionality):
    def __init__(self, n_gram, bands, rows, seed):
        super().__init__(n_gram, bands, rows, seed)
        self.lsh = None

    def compute(self):
        self.lsh = LSH(
            MinHash(
                self.original_documents,
                n_gram=self.n_gram,
                n_gram_type='term',
                permutations=self.signature_length,
                seed=self.seed
            ),
            range(len(self.original_documents)),
            no_of_bands=self.bands
        )

    def get_all_similarities(self, s: float):
        return self.lsh.edge_list(min_jaccard=s, jaccard_weighted=True)

    # to query some content, we first have to add it to our set, minhash it and than query its id..
    def query_content(self, content: str, s: float):
        doc_id = len(self.original_documents)
        self.original_documents.append(content)

        # add to set (M)
        self.lsh.update(MinHash(
            [content],
            n_gram=self.n_gram,
            n_gram_type='term',
            permutations=self.signature_length,
            seed=self.seed
        ), [doc_id])

        # query matching documents
        return self.lsh.query(doc_id, min_jaccard=s)


## 3.4 Our own implementation of LSH
Our own implementation requires some additional methods to get all the functionality. 

In [ ]:
class LSHImplementation(LSHFunctionality):
    def __init__(self, n_gram, bands, rows, seed=123):
        super().__init__(n_gram, bands, rows, seed)
        self.hash_tables = []  # a dictionary for each band
        self.M = []  # a signature for each document
        self.similarities = {}  # keys are document pairs and the values are band hits
        random.seed(seed) # prepare signature hash functions based on seed
        self.prepared_hash_functions = [HashFunction(salt=random.getrandbits(64)) for _ in range(self.signature_length)]

    # Construct M, create hash tables and compute similarities
    def compute(self):
        self.M = self.construct_M()
        self.hash_tables = self.construct_hash_tables()
        self.similarities = self.construct_similarities()

    # Create a signature for each document
    def construct_M(self):
        M = []
        for original_doc in self.original_documents:
            signature = self.doc_to_signature(original_doc)
            M.append(signature)
        return M

    # Pre process the document, shingle its contents, hash the shingles and create the signature using minhash
    def doc_to_signature(self, original_doc):
        # ["rose", "is", "a", "rose", "is", "a", "rose"]
        terms = preprocess_document(original_doc)
        # To set of shingles: {34, 727, 1, .., 934}
        hashed_shingles = self.terms_to_hashed_shingles(terms)
        signature = []
        for hash_f in self.prepared_hash_functions:
            # returns shingle for which h_i outputs the lowest value
            min_hash = min(hashed_shingles, key=hash_f.compute_int64)
            signature.append(min_hash)
        return signature  # <- sketch!

    # -> "rose is a rose is a rose"
    # -> [["rose", "is", "a"], ["is", "a", "rose"], ["a", "rose", "is"], ["rose", "is", "a"], ["is", "a", "rose"]]
    # -> [44, 24, 17, 44, 24]
    # -> {44, 24, 17}
    def terms_to_hashed_shingles(self, terms):
        hash_f = HashFunction()  # no salt
        no_shingles = len(terms) - self.n_gram + 1
        return set([hash_f.compute_strings(terms[i:i + self.n_gram]) for i in range(no_shingles)])

    # Construct a hash table (dictionary) for each band, the row values in the signature is a key in the table
    # If doc1 has values (1,2,3) for band 2, and doc2 also has values (1,2,3) for band 2,
    # then they will end up in the same bucket.
    def construct_hash_tables(self):
        bands_hash_tables = []
        for b in range(self.bands):
            hash_table = {}
            for doc_id in range(len(self.M)):
                signature = self.M[doc_id]
                key = tuple(signature[b * self.rows:(b + 1) * self.rows])
                if key in hash_table:
                    hash_table[key].append(doc_id)
                else:
                    hash_table[key] = [doc_id]
            bands_hash_tables.append(hash_table)
        return bands_hash_tables

    # Construct all similarities by keeping track of all hits between documents
    # Result -> {(doc1, doc2):5, (doc2, doc7):3}
    # If total_bands=10, then the jaccard for doc1&2 is 5/10 = 0.5
    def construct_similarities(self):
        similarities = {}
        for b in range(self.bands):
            for bucket in self.hash_tables[b].values():
                no_docs = len(bucket)
                # need at least 2 docs in a bucket to have a candidate pair
                if no_docs > 1:
                    # make all combinations between documents in bucket d(d-1)/2
                    for i in range(no_docs - 1):
                        for j in range(i + 1, no_docs):
                            candidate_pair = tuple([bucket[i], bucket[j]])
                            if candidate_pair in similarities:
                                similarities[candidate_pair] += 1
                            else:
                                similarities[candidate_pair] = 1
        return similarities

    # Get all document id's where the jaccard >= s
    def get_all_similarities(self, s: float):
        # Now the jaccard value is the amount of band hits / total_bands, but only return if >= s
        return [(doc1, doc2, hits / self.bands)
                for ((doc1, doc2), hits) in self.similarities.items() if hits / self.bands >= s]

    # Create a signature for the new document, and compare its bands with the bands hash table to find similar documents
    def query_content(self, content: str, s: float):
        similarities = {}
        signature = self.doc_to_signature(content)
        for b in range(self.bands):
            candidate_pair = tuple(signature[b * self.rows:(b + 1) * self.rows])
            if candidate_pair in self.hash_tables[b]:
                # all documents that share the same row values in band b
                for doc_id in self.hash_tables[b][candidate_pair]:
                    # keep counters how many times another doc has the same band values
                    if doc_id in similarities:
                        similarities[doc_id] += 1
                    else:
                        similarities[doc_id] = 1

        # Now the jaccard value is the amount of band hits / total_bands, but only return if >= s
        return [(doc, hits / self.bands)
                for (doc, hits) in similarities.items() if hits / self.bands >= s]



## 3.5 Simple Test

In [ ]:
# example from https://pypi.org/project/snapy/
documents = [
  'Jupiter is primarily composed of hydrogen and a quarter of its mass being helium',
  'Jupiter moving out of the inner Solar System would have allowed the formation of inner planets.',
  'A helium atom has about four times as much mass as a hydrogen atom, so the composition changes when described as the proportion of mass contributed by different atoms.',
  'Jupiter is primarily composed of hydrogen and a quarter of its mass being helium',
  'A helium atom has about four times as much mass as a hydrogen atom and the composition changes when described as a proportion of mass contributed by different atoms.',
  'Theoretical models indicate that if Jupiter had much more mass than it does at present, it would shrink.',
  'This process causes Jupiter to shrink by about 2 cm each year.',
  'Jupiter is mostly composed of hydrogen with a quarter of its mass being helium',
  'The Great Red Spot is large enough to accommodate Earth within its boundaries.'
]
# changed 'much' to 'a lot' from document 5
plagiarized_doc = 'Theoretical models indicate that if Jupiter had a lot more mass than it does at present, it would shrink.'

# test both implementations
for constructor in [LSHImplementation, LSHLibrary]:
  lsh = constructor(n_gram=2, bands=10, rows=2, seed=999)
  lsh.add_documents(documents)
  lsh.compute()
  s = 0.4
  print(f"\n========== {lsh.__class__.__name__} ==========")
  print(f"All similarities s>={s}:", lsh.get_all_similarities(s=s))
  print(f"Find similar documents to plagiarized doc with s>={s} (doc 5 expected):", lsh.query_content(plagiarized_doc, s=s))

## 3.6 Time comparison

In [ ]:
print(df.head())

for constructor in [LSHImplementation, LSHLibrary]:
  lsh = constructor(n_gram=4, bands=20, rows=4, seed=17)

  print(f"\n========== {lsh.__class__.__name__} ==========")
  print("Read CSV.. ", end='')
  time_start = time.time()
  lsh.read_csv('/content/drive/MyDrive/IR-Assignment-3/data/news_articles_small.csv')
  print(f"({round((time.time()-time_start)/60, 2)} minutes)")

  print("Construct M.. ", end='')
  time_start = time.time()
  lsh.compute()
  print(f"({round((time.time() - time_start) / 60, 2)} minutes)")

  s = 0.6
  print(f"Find all similar documents with s >= {s}")
  time_start = time.time()
  sim = lsh.get_all_similarities(s=s)
  print(f"{len(sim)} similarities found ({round((time.time() - time_start) / 60, 2)} minutes): ", sim)


# 4. Evaluation

### Prepare Some Plagiarised Documents

In [ ]:
from RandomWordGenerator import RandomWord

In [ ]:
# Replace first x words by random words
def create_plagiarised_doc_range(document, x):
  words=doc_1.split(" ")
  rw = RandomWord(max_word_size = 5,
                constant_word_size=True,
                include_digits=False,
                special_chars=r"@_!#$%^&*()<>?/\|}{~:",
                include_special_chars=False)
  for word in range(0, x):
    words[word] = rw.generate()
  return " ".join(words)

# Replace every xth word by a random word
def create_plagiarised_doc_step(document, x):
  words=doc_1.split(" ")
  rw = RandomWord(max_word_size = 5,
                constant_word_size=True,
                include_digits=False,
                special_chars=r"@_!#$%^&*()<>?/\|}{~:",
                include_special_chars=False)
  for word in range(0, len(words), x):
    words[word] = rw.generate()
  return " ".join(words)


def create_plagiarised_docs(document, duplicates):
  # Add docstring
  duplicates_dict = {}
  for i in range(1, duplicates+1):
    plagiarised_doc = create_plagiarised_doc_step(doc_1, i+1)
    duplicates_dict[f"plagiarised_doc_step_{i}"] = plagiarised_doc
  for i in range(1, duplicates+1):
    plagiarised_doc = create_plagiarised_doc_range(doc_1, i*15)
    duplicates_dict[f"plagiarised_doc_range_{i}"] = plagiarised_doc
  return duplicates_dict

In [ ]:
doc_1 = df['article'].iloc[0]
doc_1

In [ ]:
duplicates_dict = create_plagiarised_docs(doc_1, 10)
duplicates_dict

### Calculate Jaccard Similarity between Plagiarised Documents and Original Document

In [ ]:
# Preprocess doc_1
doc_1_set= set(preprocess_document(doc_1))

In [ ]:
# Calculate Jaccard Similarity between Doc_1 and its duplicates
jaccardVals = {}
for key in duplicates_dict:
  duplicate_terms = preprocess_document(duplicates_dict[key])
  duplicate_set = set(duplicate_terms)
  jaccard = len(doc_1_set.intersection(duplicate_set)) / len(doc_1_set.union(duplicate_set))
  jaccardVals[key] = jaccard

In [ ]:
# Jaccard similarity between duplicates and document 1:
jaccardVals

In [ ]:
jaccardVals_arr = np.array(list(jaccardVals.values()))
sns.histplot(jaccardVals_arr, bins=10)

### Evaluation

In [ ]:
# constructor
lsh = LSHImplementation(n_gram=2, bands=100, rows=3, seed=17)

# add documents to set (do not pre process)
# lsh.add_documents(articleList)

# or read from csv
# lsh.read_csv('/content/drive/MyDrive/IR-Assignment-3/data/news_articles_small.csv')
lsh.add_documents([doc_1]) # is faster

# create M and buckets
lsh.compute() 

# get all similarities in database
matches = lsh.get_all_similarities(s=0.4)
# get matches based on new doc
for key in duplicates_dict:
  print(key)
  print(duplicates_dict[key])
  matches = lsh.query_content(duplicates_dict[key], s=0.4)
  print(matches)

In [ ]:
# set value of M
# set value of s
for M in range(0, 1000, 100):
  for s in range(0, 100, 10):
    s = s//10

# Calculate precision

# Create precision plot